<a href="https://colab.research.google.com/github/PavelNovikov888/PDA_vectorization_text/blob/master/Chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install annoy --quiet
! pip install stop-words --quiet
! pip install pymorphy2 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 26.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import os
# методы работы со строками
import string
# Алгоритм классификации по методу поиска ближайших соседей(ANN) в Python,
# оптимизированный для использования памяти и загрузки/сохранения на диск.
import annoy
# библиотека для раскодирования файлов
import codecs

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec

import numpy as np
# выводит индикатор выполнения итератора
from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
!sudo apt-get install cadaver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libneon27-gnutls
The following NEW packages will be installed:
  cadaver libneon27-gnutls
0 upgraded, 2 newly installed, 0 to remove and 18 not upgraded.
Need to get 144 kB of archives.
After this operation, 446 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libneon27-gnutls amd64 0.32.2-1 [75.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 cadaver amd64 0.23.3-2.1build1 [69.3 kB]
Fetched 144 kB in 0s (312 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)

In [ ]:
import os

#Load file to Google Colab from WebDav
#site - WebDav host site. E.g. "webdav.yandex.ru"
#filepath - WebDav filepath. E.g. "Colab"
#filename - WebDav filename. E.g. "creditcard.csv.zip"
#login - WebDav login
#password - WebDav password
def loadFileFromWebDav(site, filepath, filename, login, password):
  if os.path.isfile(filename):
    return

  netrc = os.environ['HOME'] + "/.netrc"

  if not os.path.isfile(netrc): #Check if .netrc file for WebDav access exists
    #!rm $netrc
    netrcText = "machine " + site + " login " + login + " password " + password
    !echo $netrcText > $netrc
    !chmod 600 $netrc
    !cat $netrc

  result = !cadaver --version #Check if WebDav console client cadaver is installed
  if "command not found" in result:
    !sudo apt-get install cadaver

  result = !cadaver --version
  if not ("command not found" in result): #Create file to get the file from WebDav
    #!rm .cadaverrc
    cadaverrc = "cd " + filepath + "\r\n"
    cadaverrc += "get " + filename + "\r\n"
    cadaverrc += "exit\r\n"

    file1 = open("cadaverrc","w")
    file1.write(cadaverrc)
    file1.close()
    !cat "cadaverrc"

    webdavurl = "https://" + site
    !cadaver -r cadaverrc $webdavurl

    if ".zip" in filename: #Unpack file if it was compressed by zip
      !unzip $filename

In [ ]:
# !rm .cadaverrc
!rm Otvety.txt
loadFileFromWebDav("webdav.yandex.ru", "Colab", "Otvety.txt", "pavelzrenie", "nuagieinlgexdsbi")

rm: cannot remove 'Otvety.txt': No such file or directory
machine webdav.yandex.ru login pavelzrenie password nuagieinlgexdsbi
cd Colab
get Otvety.txt
exit
Connection to `webdav.yandex.ru' closed.


In [ ]:
!rm ProductsDataset.zip
!rm ProductsDataset.csv
loadFileFromWebDav("webdav.yandex.ru", "Colab", "ProductsDataset.zip", "pavelzrenie", "nuagieinlgexdsbi")

rm: cannot remove 'ProductsDataset.zip': No such file or directory
rm: cannot remove 'ProductsDataset.csv': No such file or directory
cd Colab
get ProductsDataset.zip
exit
Progress: [=============================>] 100.0% of 4398055 bytes succeeded.
Connection to `webdav.yandex.ru' closed.
Archive:  ProductsDataset.zip
  inflating: ProductsDataset.csv     


In [ ]:
f = 0
with codecs.open("Otvety.txt", "r", "utf-8") as fin:
      # из исходного файла перебираем строки
      for line in fin:
        print(line)
        f += 1
        if f == 30:
          break



---

вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО ВАМ здесь кого советовали завести?)) . 

хомячка.... 

мужика, йопаря, собачку и 50 кошек))). 

Общение !). 

паучка. 

Да пол мне бы памыть!<br>А таг то ни чо. Типа ни каво!. 

я тут вообще что бы пообщаться..... 

А мне советовали сиси завести...))). 

Ну, слава богу, мужика завести ещё не советовали))) А вот сватать к кому только не сватали). 

мне тут советовали завести любовника, мужа и много кошек )))<br>приветик ))). 

---

Как парни относятся к цветным линзам? Если у девушки то зеленые глаза, то голубые...)) . 

меня вобще прикалывает эта тема :). 

когда этобыло редкость - было забавно, а когда все знают, что эта фальшивка, то уже не прикольно, как силиконовые сиськи или как налепленные синтетические волосы. 

---

Что делать, сегодня нашёл 2 миллиона рублей? . 

Если это "счастье " действительно на вас свалилось, лучше пойти в милицию и заявить о находке. Такие деньги просто так не терют, а что самое интересное их непримен

Предобработаем ответы mail.ru из файла: к каждому вопросу присоединим 1 ответ и запишем в файл на будущее. Это позволит нам сэкономить время и ресурсы при дальнейшем препроцессинге текста

In [ ]:
# вопрос не написан
question = None
# ответ не написан
written = False

# записываем в файл подготовленные ответы(prepared_answers.txt)
with codecs.open("prepared_answers.txt","w", "utf-8") as fout:
  # читаем файл Otvety.txt
    with codecs.open("Otvety.txt", "r", "utf-8") as fin:
      # из исходного файла перебираем строки
        for line in tqdm(fin):
          # если строка начинается с ---
            if line.startswith("---"):
                # ответ не написан
                written = False
                # переходим к следующей строке
                continue
            # если строка не начинается с "---", ответ не написан
            # и у нас есть ответ на вопрос
            if not written and question is not None:
                # записываем строку в файл prepared_answers.txt текст: вопрос+tab+ответ
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                # ответ написан
                written = True
                # вопрос не написан
                question = None
                # идем к следующей строке
                continue
            # если строка не --- , ответ не написан
            if not written:
                # значит это вопрос
                question = line.strip()
                # переходим к следующей строке
                continue

0it [00:00, ?it/s]

In [ ]:
f = 0
with codecs.open("prepared_answers.txt", "r", "utf-8") as fin:
      # из исходного файла перебираем строки
      for line in fin:
        print(line)
        f += 1
        if f == 3:
          break

	вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО ВАМ здесь кого советовали завести?)) . 

Как парни относятся к цветным линзам? Если у девушки то зеленые глаза, то голубые...)) .	меня вобще прикалывает эта тема :). 

Что делать, сегодня нашёл 2 миллиона рублей? .	Если это "счастье " действительно на вас свалилось, лучше пойти в милицию и заявить о находке. Такие деньги просто так не терют, а что самое интересное их неприменно будут искать и поверьте мне найдут, видел подобное в жизни. Можно нарваться на бабушку конечно, которая хотела помоч внуку с покупкой квартиры, а можно на бандитов, которые будут с вами разговаривать иначе чем бабушка с милицией. Выбор за вами, есть еще конечно шанс, что это подарок с выше за котрый с вас никто не спросит, тогда лучше отдать хотябы 500 на благотворительность. дабы не спугнуть удачу!. 

Эбу в двенашке называется Итэлма что за эбу? .	ЭБУ — электронный блок управления двигателем автомобиля, его другое название — контроллер. Он принимает информацию от 

In [ ]:
f = 0
with codecs.open("prepared_answers.txt", "r", "utf-8") as fin:
      # из исходного файла перебираем строки
      for line in fin:
        print(line.split("\t")[0])
        f += 1
        if f == 30:
          break


Как парни относятся к цветным линзам? Если у девушки то зеленые глаза, то голубые...)) .
Что делать, сегодня нашёл 2 миллиона рублей? .
Эбу в двенашке называется Итэлма что за эбу? .
академия вампиров. сколько на даный момент частей книги академия вампиров? .
как защититься от энергетического вампира .
Кто выращивает магнолию в открытом грунте в Средней Полосе России ? Какой именно у Вас вид и сорт ? Зимует с укрытием или без укрытия ? Как цветет ? Сколько лет Вашей магнолии ? Каковы размеры деревьев ? Какие особенности выращивания ? Пожалуйста, указывайте в ответе Ваш регион и, если можно, прикрепляйте фото..
Как отформатировать диск C из доса если он в формате NTFS при команде fdisk - жесткие диски не найдены .
Как вы относитесь к парикмахерам, работающим на дому? Помешает ли вам например присутствие дома ребенка? Или мужа. Нормально ли для вас мыть голову в ванной "раком"?.
Кошку Олега не видели)? .
Почему пьяному везет больше чем трезвому? .
Как остановить выпадение волос при бере

Теперь нам нужно предобработать текст, чтобы обучить word2vec и получить эмбеддинги. Удаляем знаки препинания и делаем лемматизацию

In [ ]:
def preprocess_txt(line):
    """Предобработка текста

    Args:
        line (string): обрабатываемый текст

    Returns:
        string: Очищенный от знаков препинания и стоп-слов текст, лемматизированный
                и приведенный к нормальной форме
    """
    # очистка от знаков препинания
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    # принимаем слово (обязательно в нижнем регистре) и возвращаем нормальную форму
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    # удаляем стоп-слова
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [ ]:
# создадим список предложений
sentences = []
# создадим экземпляр класса для лемматизации
morpher = MorphAnalyzer()
# создадим множество стоп-слов
sw = set(get_stop_words("ru"))
# создадим множество со знаками пунктуации
exclude = set(string.punctuation)
# счетчик количества обрабатываемых пар вопрос-ответ
c = 0

# открываем файл с необработанными данными вопрос-ответ
with codecs.open("Otvety.txt", "r", "utf-8") as fin:
    # используем визуализацию процесса итерации строк
    for line in tqdm(fin):
      # преодобработаем тексты вопросов-ответов
        spls = preprocess_txt(line)
        # запишем обработанные строки в список
        sentences.append(spls)
        c += 1
        # ограничим количество предобработанных строк до 500000
        if c > 500000:
            break

0it [00:00, ?it/s]

In [ ]:
print(*sentences[0:20],sep='\n')

[]
[]
['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести']
['хомячок']
['мужик', 'йопарить', 'собачка', '50', 'кошка']
['общение']
['паучок']
['пол', 'памытьbr', 'таг', 'тип', 'каво']
['вообще', 'пообщаться']
['советовать', 'сися', 'завести']
['слава', 'бог', 'мужик', 'завести', 'советовать', 'сватать', 'сватать']
['советовать', 'завести', 'любовник', 'муж', 'кошка', 'brприветик']
[]
['парень', 'относиться', 'цветной', 'линза', 'девушка', 'зелёный', 'глаз', 'голубой']
['вобщий', 'прикалывать', 'тема']
['этобыть', 'редкость', 'забавно', 'знать', 'фальшивка', 'прикольно', 'силиконовый', 'сиська', 'налепить', 'синтетический', 'волос']
[]
['делать', 'найти', '2', 'миллион', 'рубль']
['счастие', 'свалиться', 'хороший', 'пойти', 'милиция', 'заявить', 'находка', 'деньга', 'тероть', 'самый', 'интересный', 'неприменный', 'искать', 'поверьте', 'найти', 'видеть', 'подобный', 'нарваться', 'бабушка', 'помочий', 'внук', 'покупка', 'квартира', 'бандит', 'разговаривать', 'иначе', 'бабушка',

In [ ]:
sentences[1]

[]

In [ ]:
# Обучим модель word2vec на наших вопросах
# оставим в списке предложений те, в которых более 2 символов
sentences = [i for i in sentences if len(i) > 2]
# создадим модель для получения векторных представлений слов
# размер векторов слов = 100,
# игнорировать слова с частотой менее 1,
# максимальное расстояние между текущим и предсказанным словом в предложении = 5
model = Word2Vec(sentences=sentences, vector_size=100, min_count=1, window=5)

model.save("w2v_model")

Теперь нам нужно сложить в индекс все вопросы. Используем библиотеку annoy. Проходимся по всем ответам, здесь нам пригодится предобработанный файл. Для создания индекса считаем, что вектор предложения - сумма word2vecов слов, которые входят в него (конечно же усредненная)

In [ ]:
# Количество признаков (размерностей) хранимого векторa = 100
# Метрика расстояния между векторами («угловой»)
index = annoy.AnnoyIndex(100 ,'angular')

# словарь, в котором будет находится индекс
index_map = {}
# счетчик элементов индекса
counter = 0

# для создания индекса используем предобработанный в формат вопрос-ответ исходный файл
with codecs.open("prepared_answers.txt", "r", "utf-8") as f:
    # проходим по строкам предобработанного файла
    for line in tqdm(f):
      # счетчик количества слов с вектором в вопросе
        n_w2v = 0
        # делим строку предобработанного файла на вопрос
        # и ответ по символу табуляции, проставленному при обработке
        spls = line.split("\t")
        # по очереди заполняем словарь(номер вопроса-ответ на вопрос)
        index_map[counter] = spls[1]
        # производим предобработку вопроса
        question = preprocess_txt(spls[0])
        # создаем нулевую матрицу(100,1) из 100 элементов
        vector = np.zeros(100)
        # для каждого слова в предобработанном вопросе
        for word in question:
          # если даннное слово присутствует в списке созданных векторов
            if word in model.wv:
              # суммируем полученные вектора слов
                vector += model.wv[word]
              # увеличиваем счетчик количества слов в вопросе
                n_w2v += 1
      # если в вопросе есть слова
        if n_w2v > 0:
          # вычисляем средний вектор
            vector = vector / n_w2v
      # записываем в индекс номер и средний вектор
        index.add_item(counter, vector)
      # увеличиваем счетчик индекса
        counter += 1
# количество деревьев в лесу(чем больше, тем точнее)
index.build(10)
# сохраним полученный индекс в файл для дальнейшего использования
index.save('speaker.ann')


0it [00:00, ?it/s]

True

In [ ]:
import json

In [ ]:
with open('index_map', 'w') as convert_file:
     convert_file.write(json.dumps(index_map))

In [ ]:
!zip -r /content/index_map.zip /content/index_map

  adding: content/index_map (deflated 85%)


In [ ]:
from google.colab import files
files.download('/content/index_map.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(model.wv['кот'], model.wv['парень'])


[-1.55950570e+00  1.59032655e+00  6.56097054e-01  1.13037431e+00
  1.07923746e+00 -1.28520250e+00  9.33649659e-01  5.17602026e-01
 -1.06544530e+00 -1.68133235e+00 -3.63310650e-02 -1.67546225e+00
 -2.23981428e+00 -1.42771626e+00  1.12234509e+00 -1.73416436e-01
 -2.99072266e-01 -1.96689916e+00  7.02841699e-01 -2.13585210e+00
 -7.48486340e-01 -1.17823219e+00 -3.80826324e-01 -1.47419143e+00
  3.72456610e-01  1.58719778e-01 -2.24195433e+00  2.79980898e-01
 -2.31517887e+00  8.80648717e-02  1.44514120e+00  2.17594051e+00
 -1.32167673e+00 -5.12817085e-01  9.36782241e-01  2.89658785e-01
 -5.44820309e-01  4.17454451e-01 -1.25084341e+00 -8.94260257e-02
 -4.80901837e-01  5.75526059e-01 -1.42102957e+00  5.18914700e-01
  1.66541541e+00 -1.44180691e+00 -1.80855656e+00  5.47207296e-01
 -5.28336763e-01  4.77529645e-01  4.37306240e-02 -5.71301937e-01
 -1.50789273e+00 -1.11805402e-01 -2.63015652e+00 -1.21950924e-01
  3.27736676e-01  3.30730830e-03  1.07138801e+00  1.63164473e+00
  5.08890867e-01  1.07389

In [ ]:
dr = model.wv.most_similar(positive=["кот","парень"])
print(*dr, sep='\n')

('девчёнка', 0.877497136592865)
('кобель', 0.8670281767845154)
('подружка', 0.8656561970710754)
('котик', 0.864730179309845)
('сука', 0.8599978685379028)
('пацан', 0.8563041687011719)
('ласково', 0.8546333312988281)
('подруга', 0.8534715175628662)
('комфортно', 0.8534315228462219)
('ныть', 0.8530231714248657)


In [ ]:
dr = model.wv.most_similar(positive=["кошка","дом"])
print(*dr, sep='\n')

('кот', 0.8593893051147461)
('собака', 0.8492904901504517)
('кролик', 0.838685154914856)
('котёнок', 0.8363857865333557)
('щенок', 0.8363141417503357)
('крыса', 0.8287786841392517)
('метить', 0.8238303065299988)
('питомец', 0.8221099972724915)
('хозяин', 0.8198133111000061)
('приют', 0.8190529346466064)


# Новый раздел

Теперь остается реализовать метод, который получит на вход вопрос и найдет ответ к нему!
Мы препроцессим вопрос, находим ближайший вопрос и выбираем ответ на ближайший вопрос.

In [ ]:
def find_answer(question):

   """Функция реализует метод, который получает на вход вопрос и находит ответ к нему.
      Для этого вопрос  препроцессится, далее находится ближайший вопрос и
      происходит выбор ответа на ближайший вопрос.

    Args:
        question (string): заданный вопрос

    Returns:
        string: найденный ответ
    """
    # готовим для обработки с помощью препроцессинга заданный вопрос
   preprocessed_question = preprocess_txt(question)
    # счетчик слов в вопросе
   n_w2v = 0
    # создаем вектор
   vector = np.zeros(100)
    # определяем сумму векторов слов входящих в вопрос
    # и количество слов входящих в вопрос
   for word in preprocessed_question:
       if word in model.wv:
          vector += model.wv[word]
          n_w2v += 1
  # определяем средний вектор соответствующий заданному вопросу
   if n_w2v > 0:
      vector = vector / n_w2v
  # ищем 1 ближайший вектор заданного вопроса и вопроса из индекса
   answer_index = index.get_nns_by_vector(vector, 1)
    # возвращаем ответ из наиболее близкого заданному вопроса
   return index_map[answer_index[0]]

In [ ]:
gg = find_answer('есть ли кот')

In [ ]:
print(gg)

Коты в Кошках.. 



In [ ]:
products = pd.read_csv('/content/ProductsDataset.zip')

In [ ]:
products['content_type'] = 1

→ Обучен классификатор «товарный запрос vs. болталка»

Осуществлён препроцессинг текста (как минимум удаление знаков препинания, приведение к нижнему регистру, стемминг/лемматизация).
Текст векторизирован любым из изученных способов (CountVectorizer, TfidfVectorizer, HashingVectorizer, Word2Vec).
Выборка разделена на обучающую и валидационную.
Обучен классификатор с расчётом метрик на валидации (любое семейство алгоритмов, но предпочтительнее просто логистическая регрессия).
Модель сохранена и при применении загружается из pkl-файла (или аналога).

In [ ]:
products.dropna(inplace = True)

In [ ]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33531 entries, 0 to 35546
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           33531 non-null  object 
 1   descrirption    33531 non-null  object 
 2   product_id      33531 non-null  object 
 3   category_id     33531 non-null  float64
 4   subcategory_id  33531 non-null  object 
 5   properties      33531 non-null  object 
 6   image_links     33531 non-null  object 
 7   content_type    33531 non-null  int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 2.3+ MB


In [ ]:
products['title'] = products['title'] + products['descrirption']

In [ ]:
# products['preprocess_txt'] = products['title'].apply(lambda x: preprocess_txt(x))

In [ ]:
dict_questions = {}
with codecs.open("prepared_answers.txt", "r", "utf-8") as fin:
      # из исходного файла перебираем строки
      for line in fin:
        dict_questions[line.split("\t")[0]] = 0

In [ ]:
content_dataset = pd.DataFrame(list(dict_questions. items ()), columns = ['title','content_type'])

In [ ]:
content_dataset.head()

,title,content_type
0,,0
1,Как парни относятся к цветным линзам? Если у д...,0
2,"Что делать, сегодня нашёл 2 миллиона рублей? .",0
3,Эбу в двенашке называется Итэлма что за эбу? .,0
4,академия вампиров. сколько на даный момент час...,0


In [ ]:
content_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1158610 entries, 0 to 1158609
Data columns (total 2 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   title         1158610 non-null  object
 1   content_type  1158610 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 17.7+ MB


In [ ]:
classification_dataset = pd.concat([content_dataset, products[['title','content_type']]])

In [ ]:
classification_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1192141 entries, 0 to 35546
Data columns (total 2 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   title         1192141 non-null  object
 1   content_type  1192141 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 27.3+ MB


In [ ]:
classification_dataset['title'] = classification_dataset['title'].apply(lambda x: preprocess_txt(x))

NameError: ignored

In [ ]:
classification_dataset.head()

In [ ]:
print(round(df2['content_type'].value_counts(normalize = True)*100,2))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
X_train_cs, X_valid, y_train_cs, y_valid = train_test_split(df2['title'],
                                                                            df2['content_type'],
                                                                            stratify=df2['content_type'],
                                                                            test_size=0.2,
                                                                            random_state=42)
print('Train:\n', y_train_cs.value_counts(normalize=True), sep='')
print('Valid:\n', y_valid.value_counts(normalize=True), sep='')

In [ ]:
# создадим и обучим объект векторизации TfidfVectorizer
vectorizer  = TfidfVectorizer()
values = vectorizer.fit(df2['title'])


In [ ]:
# 1.конвертируем тексты(обучающий и тестовый) в векторы tf-idf
# 2 конвертируем метки класса в массивы numpy
X_train_cs = values.transform(X_train_cs)

X_valid = values.transform(X_valid)
